In [0]:
def spark_connector():
    spark.conf.set("fs.azure.account.key.saadlsoptum.dfs.core.windows.net",dbutils.secrets.get(scope='optumscope',key='adlskey'))
    return "connected to adls" 


In [0]:
def read_bronze_csv_data(fn):
    df=spark.read.csv("abfss://optum@saadlsoptum.dfs.core.windows.net/Bronze/"+fn+".csv",header=True,inferSchema=True)
    return df

In [0]:
#Generic code to read file from silver layer
def read_silver_csv_data(fn):
    df=spark.read.csv("abfss://optum@saadlsoptum.dfs.core.windows.net/Silver/"+fn+".csv",header=True,inferSchema=True)
    return df

In [0]:
def read_bronze_json_data(fn):
    df=spark.read.json("abfss://optum@saadlsoptum.dfs.core.windows.net/Bronze/"+fn+".json")
    return df

In [0]:
import datetime, uuid

def write2silver(df, file_name):
    silver_path = "abfss://optum@saadlsoptum.dfs.core.windows.net/Silver/"
    
    # unique temp folder for this run
    unique_id = datetime.datetime.now().strftime("%Y%m%d%H%M%S") + "_" + str(uuid.uuid4())
    temp_path = f"{silver_path}/output_temp/{unique_id}/"
    
    # always a single CSV file at final path
    final_file = f"{silver_path}/{file_name}.csv"
    
    # write to temp
    df.coalesce(1).write.mode("overwrite").option("header", True).csv(temp_path)
    
    # pick actual CSV file from Spark output
    files = dbutils.fs.ls(temp_path)
    csv_files = [f.path for f in files if f.name.endswith(".csv")]
    
    if not csv_files:
        raise Exception("No CSV file found in temp path")
    
    csv_file = csv_files[0]
    
    # move & overwrite final
    dbutils.fs.mv(csv_file, final_file, True)
    
    # cleanup temp folder
    dbutils.fs.rm(temp_path, recurse=True)
    
    print(f"✅ File successfully written to Silver: {final_file}")


In [0]:
import datetime, uuid

def write2gold(df, file_name):
    gold_path = "abfss://optum@saadlsoptum.dfs.core.windows.net/Gold/"
    
    # Unique temp folder for staging
    unique_id = datetime.datetime.now().strftime("%Y%m%d%H%M%S") + "_" + str(uuid.uuid4())
    temp_path = f"{gold_path}/output_temp/{unique_id}/"
    
    # Final single CSV file
    final_file = f"{gold_path}/{file_name}.csv"
    
    # Write DataFrame to temp path
    df.coalesce(1).write.mode("overwrite").option("header", True).csv(temp_path)
    
    # Pick Spark’s actual CSV file
    files = dbutils.fs.ls(temp_path)
    csv_file = [f.path for f in files if f.name.endswith(".csv")][0]
    
    # Move & overwrite final file
    dbutils.fs.mv(csv_file, final_file, True)
    
    # Cleanup temp folder
    dbutils.fs.rm(temp_path, recurse=True)
    
    print(f"✅ File successfully written to Gold: {final_file}")
